# RAG - Vector Search with Qdrant

```sh
docker run --rm --name qdrant\
     -p 6333:6333 \
     -p 6334:6334 \
     -v "${PWD}/qdrant_storage:/qdrant/storage:z" \
     qdrant/qdrant:v1.16.3
```

Go to http://localhost:6333/dashboard


Install packages


In [1]:
!uv pip install -q \
    requests==2.32.5 \
    python-dotenv==1.2.1 \
    litellm==1.78.5 \
    qdrant-client==1.16.2 \
    fastembed==0.7.4

Import packages


In [ ]:
import json
import random
from typing import Any, Dict, List

import litellm
import requests
from dotenv import load_dotenv
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models

load_dotenv()

True

Download documents


In [ ]:
docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]
    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

Create a Qdrant client instance


In [ ]:
qd_client = QdrantClient("http://localhost:6333")

Verify models compatible with current setup


In [ ]:
EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

Choose model


In [ ]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

Define a collection


In [ ]:
collection_name = "zoomcamp-rag"

if qd_client.collection_exists(collection_name=collection_name):
    qd_client.delete_collection(collection_name=collection_name)

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY, distance=models.Distance.COSINE
    ),
)

True

Create points


In [ ]:
points = []
id = 0

for course in documents_raw:
    for doc in course["documents"]:
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc["text"], model=model_handle),
            payload={
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"],
            },
        )
        points.append(point)

        id += 1

Generate embeddings and insert into Qdrant


In [ ]:
qd_client.upsert(collection_name=collection_name, points=points)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
def search(query, limit=1):
    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(text=query, model=model_handle),
        limit=limit,
        with_payload=True,
    )

    return results

In [ ]:
course = random.choice(documents_raw)
course_piece = random.choice(course["documents"])
print(json.dumps(course_piece, indent=2))

{
  "text": "Validation dataset helps to validate models and prediction on unseen data. This helps get an estimate on its performance on fresh data. It helps optimize the model.\nEdidiong Esu\nBelow is an extract of Alexey's book explaining this point. Hope is useful\nWhen we apply the fit method, this method is looking at the content of the df_train dictionaries we are passing to the DictVectorizer instance, and fit is figuring out (training) how to map the values of these dictionaries. If categorical, applies one-hot encoding, if numerical it will leave it as it is.\nWith this context, if we apply the fit to the validation model, we are \"giving the answers\" and we are not letting the \"fit\" do its job for data that we haven't seen. By not applying the fit to the validation model we can know how well it was trained.\nBelow is an extract of Alexey's book explaining this point.\nHumberto Rodriguez\nThere is no need to initialize another instance of dictvectorizer after fitting it on 

In [ ]:
result = search(course_piece["question"])
result

QueryResponse(points=[ScoredPoint(id=529, version=1, score=0.86013603, payload={'text': 'Validation dataset helps to validate models and prediction on unseen data. This helps get an estimate on its performance on fresh data. It helps optimize the model.\nEdidiong Esu\nBelow is an extract of Alexey\'s book explaining this point. Hope is useful\nWhen we apply the fit method, this method is looking at the content of the df_train dictionaries we are passing to the DictVectorizer instance, and fit is figuring out (training) how to map the values of these dictionaries. If categorical, applies one-hot encoding, if numerical it will leave it as it is.\nWith this context, if we apply the fit to the validation model, we are "giving the answers" and we are not letting the "fit" do its job for data that we haven\'t seen. By not applying the fit to the validation model we can know how well it was trained.\nBelow is an extract of Alexey\'s book explaining this point.\nHumberto Rodriguez\nThere is no

Index fields that will be used as filters


In [ ]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword",
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

Search with filters


In [ ]:
def search_in_course(query, course, limit=1):
    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(text=query, model=model_handle),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="course", match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit,
        with_payload=True,
    )

    return results

Apply filters in search


In [ ]:
for course in [
    "data-engineering-zoomcamp",
    "machine-learning-zoomcamp",
    "mlops-zoomcamp",
]:
    print(
        course + "\n",
        search_in_course(
            "What if I submit homeworks late?",
            course=course,
        )
        .points[0]
        .payload["text"],
    )

data-engineering-zoomcamp
 No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y
Older news:[source1] [source2]
machine-learning-zoomcamp
 Depends on whether the form will still be open. If you're lucky and it's open, you can submit your homework and it will be evaluated. if closed - it's too late.
(Added by Rileen Sinha, based on answer by Alexey on Slack)
mlops-zoomcamp
 Please choose the closest one to your answer. Also do not post your answer in the course slack channel.


## RAG


In [ ]:
collection_name = "zoomcamp-faq"

if qd_client.collection_exists(collection_name=collection_name):
    qd_client.delete_collection(collection_name=collection_name)

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY, distance=models.Distance.COSINE
    ),
)

True

In [ ]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword",
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
points = []

for i, doc in enumerate(documents):
    text = doc["question"] + " " + doc["text"]
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(id=i, vector=vector, payload=doc)
    points.append(point)

In [ ]:
qd_client.upsert(collection_name=collection_name, points=points)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
def vector_search(question):
    course = "data-engineering-zoomcamp"
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(text=question, model=model_handle),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="course", match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True,
    )

    results = []

    for point in query_points.points:
        results.append(point.payload)

    return results

In [ ]:
def build_prompt(query: str, search_results: List[Dict[str, Any]]):
    prompt_template = """
    You're a course teaching assistant. Answer the question based on the CONTEXT.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """
    context = ""

    for doc in search_results:
        context = (
            context
            + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        )

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [ ]:
def llm(prompt: str):
    messages = [
        {
            "role": "user",
            "content": prompt,
        },
    ]

    completion = litellm.completion(
        model="gemini/gemini-2.5-flash",
        messages=messages,
    )

    return completion.choices[0].message.content

In [ ]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
print(rag("how do I run kafka?"))

To run Kafka components, the context provides several methods:

*   **For Java producers/consumers/kstreams:** In the project directory, run:
    `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`

*   **For Python producers (e.g., producer.py):**
    1.  Create a virtual environment (if not already done): `python -m venv env`
    2.  Activate the environment: `source env/bin/activate` (for MacOS/Linux, or `env/Scripts/activate` for Windows)
    3.  Install requirements: `pip install -r ../requirements.txt`
    4.  Then run your Python file within this activated environment.
    Remember to activate the virtual environment every time you need to use it.

**To ensure the Kafka broker (the underlying service) is running:**
If you encounter a `kafka.errors.NoBrokersAvailable` error, it likely means your Kafka broker docker container is not working. You can start all instances by navigating to the docker compose yaml file folder and running `d